In [1]:
# Modisco
%matplotlib inline

import modisco
import json
import os.path
import pandas as pd
import numpy as np
import h5py
from metrics import Pearson
from collections import OrderedDict, Counter, defaultdict
from keras.utils import to_categorical
from keras.models import load_model
from utils import prepare_data
from deeplift.visualization import viz_sequence



2023-07-16 13:33:29.636242: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def one_hot_emb(data: pd.DataFrame) -> pd.DataFrame:
    mapping = {
        'A': 0,
        'C': 1,
        'G': 2,
        'T': 3
    }
    one_hot_encode_lam = lambda seq: to_categorical([mapping[x] for x in seq])
    return data.apply(one_hot_encode_lam)

In [10]:
train_data, valid_data, test_data = prepare_data()

max_seq_len = train_data['seq'].apply(lambda x: len(x)).max()

max_seq_len = 20788

train_data['length'] = train_data.seq.str.len()

train_data = train_data[train_data.length <= max_seq_len]

data_one_hot = one_hot_emb(train_data['seq'])
#data_struct = train_data['struct']

#padded_sequences = np.zeros((len(train_data['seq']), max_seq_len, 6), dtype=np.float32)
padded_sequences = np.zeros((len(train_data['seq']), max_seq_len, 4), dtype=np.float32)

indices = np.arange(train_data.shape[0])

for i, idx in enumerate(indices):
    #tmp = data_struct.iloc[idx]
    #padded_mask_struct = np.expand_dims(np.array(tmp != 'nan'), axis=1)
    #tmp[tmp == 'nan'] = -1
    #padded_struct = np.expand_dims(tmp, axis=1).astype('float64')
    seq_data = data_one_hot.iloc[idx]
    #seq_data = np.concatenate([seq_data, padded_mask_struct, padded_struct], axis=1)
    padded_sequences[i, -len(data_one_hot.iloc[idx]):, :] = seq_data

data_one_hot = padded_sequences

In [ ]:
"""
    Integrated Gradients
"""
from motif_search import motif_utilities


model_name = "moitfTest3"

model = load_model(f'model_outputs/{model_name}.keras',compile=False)

seqs_to_explain = data_one_hot

gradients = motif_utilities.get_gradients(model, seqs_to_explain)

2023-07-16 13:41:55.708201: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-16 13:41:55.710241: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-16 13:41:55.713270: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
print(gradients)
# print(data_one_hot)

viz_sequence.plot_weights(data_one_hot[0][:20], subticks_frequency=20)

# motif_utilities.plot_weights(data_one_hot[0][:20])

In [ ]:
#Uncomment to refresh modules for when tweaking code during development:
from importlib import reload
reload(modisco.util)
reload(modisco.pattern_filterer)
reload(modisco.aggregator)
reload(modisco.core)
reload(modisco.seqlet_embedding.advanced_gapped_kmer)
reload(modisco.affinitymat.transformers)
reload(modisco.affinitymat.core)
reload(modisco.affinitymat)
reload(modisco.cluster.core)
reload(modisco.cluster)
reload(modisco.tfmodisco_workflow.seqlets_to_patterns)
reload(modisco.tfmodisco_workflow)
reload(modisco)

scores = gradients
hypothetical_scores = [] # optional

tasks = {
    "task0": "Identify high-importance windows of the sequences, termed seqlets",
    "task1": "Cluster recurring similar seqlets into motifs",
    "task2": "Scan through importance scores across the genome to call motif instances (AKA hit scoring)"
}





null_per_pos_scores = modisco.coordproducers.LaplaceNullDist(num_to_samp=5000)
tfmodisco_results = modisco.tfmodisco_workflow.workflow.TfModiscoWorkflow(
                    #Slight modifications from the default settings
                    sliding_window_size=15,
                    flank_size=5,
                    target_seqlet_fdr=0.15,
                    seqlets_to_patterns_factory=
                     modisco.tfmodisco_workflow.seqlets_to_patterns.TfModiscoSeqletsToPatternsFactory(
                        #Note: as of version 0.5.6.0, it's possible to use the results of a motif discovery
                        # software like MEME to improve the TF-MoDISco clustering. To use the meme-based
                        # initialization, you would specify the initclusterer_factory as shown in the
                        # commented-out code below:
                        #initclusterer_factory=modisco.clusterinit.memeinit.MemeInitClustererFactory(
                        #    meme_command="meme", base_outdir="meme_out",
                        #    max_num_seqlets_to_use=10000, nmotifs=10, n_jobs=1),
                        trim_to_window_size=15,
                        initial_flank_to_add=5,
                        final_flank_to_add=5,
                        final_min_cluster_size=60,
                        #use_pynnd=True can be used for faster nn comp at coarse grained step
                        # (it will use pynndescent), but note that pynndescent may crash
                        #use_pynnd=True,
                        n_cores=10)
                )(
                 task_names=["task0"],#, "task1", "task2"],
                 contrib_scores=scores,
                 one_hot=data_one_hot,
                 null_per_pos_scores=null_per_pos_scores)